In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

# Load the MiDaS model from torch.hub
def load_midas_model():
    model = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
    model.eval()  # Set the model to evaluation mode
    return model

# Preprocess the image
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform(image)

# Generate depth map using MiDaS
def generate_depth_map(model, image):
    with torch.no_grad():
        input_image = preprocess_image(image).unsqueeze(0)
        depth_map = model(input_image)
        depth_map = torch.nn.functional.interpolate(depth_map.unsqueeze(1), size=(256, 256), mode='bilinear', align_corners=False)
        depth_map = depth_map.squeeze().cpu().numpy()
    return depth_map

# Define a simple classifier to distinguish between real and spoofed faces
def create_classifier():
    classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(256 * 256, 512),  # Assuming depth map size of 256x256
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.ReLU(),
        nn.Linear(128, 2)  # Two classes: real (0) and spoof (1)
    )
    return classifier

# Train the classifier
def train_classifier(classifier, dataloader, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=lr)

    classifier.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images = images.float()
            labels = labels.long()

            optimizer.zero_grad()
            outputs = classifier(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')

# Example usage
def main():
    # Paths to your images and corresponding labels (0 for real, 1 for spoof)
    image_paths = ["path_to_real_image.jpg", "path_to_spoof_image.jpg"]
    labels = [0, 1]  # Assuming the first image is real and the second is a spoof

    # Load MiDaS model
    midas_model = load_midas_model()

    # Generate depth maps for the images
    depth_maps = []
    for path in image_paths:
        image = cv2.imread(path)
        depth_map = generate_depth_map(midas_model, image)
        depth_maps.append(depth_map)

    # Convert depth maps and labels to PyTorch tensors
    depth_maps = np.array(depth_maps)
    labels = np.array(labels)

    # Convert depth maps and labels to PyTorch tensors
    depth_maps_tensor = torch.tensor(depth_maps).float()
    labels_tensor = torch.tensor(labels).long()

    # Create a dataset and dataloader
    dataset = TensorDataset(depth_maps_tensor, labels_tensor)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    # Initialize the classifier
    classifier = create_classifier()

    # Train the classifier
    train_classifier(classifier, dataloader, epochs=10)

    # Save the trained classifier
    torch.save(classifier.state_dict(), "depth_classifier.pth")

if __name__ == "__main__":
    main()


Using cache found in C:\Users\Kavyansh Tyagi/.cache\torch\hub\intel-isl_MiDaS_master


ModuleNotFoundError: No module named 'midas.dpt_depth'; 'midas' is not a package

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from midas.dpt_depth import DPTDepthModel
from midas.transforms import Resize, NormalizeImage, PrepareForNet

SyntaxError: invalid syntax (2420262892.py, line 9)

In [10]:
import torch
import torch.nn as nn

from .base_model import BaseModel
from .blocks import FeatureFusionBlock, Interpolate, _make_encoder


class MidasNet(BaseModel):
    """Network for monocular depth estimation.
    """

    def __init__(self, path=None, features=256, non_negative=True):
        """Init.

        Args:
            path (str, optional): Path to saved model. Defaults to None.
            features (int, optional): Number of features. Defaults to 256.
            backbone (str, optional): Backbone network for encoder. Defaults to resnet50
        """
        print("Loading weights: ", path)

        super(MidasNet, self).__init__()

        use_pretrained = False if path is None else True

        self.pretrained, self.scratch = _make_encoder(backbone="resnext101_wsl", features=features, use_pretrained=use_pretrained)

        self.scratch.refinenet4 = FeatureFusionBlock(features)
        self.scratch.refinenet3 = FeatureFusionBlock(features)
        self.scratch.refinenet2 = FeatureFusionBlock(features)
        self.scratch.refinenet1 = FeatureFusionBlock(features)

        self.scratch.output_conv = nn.Sequential(
            nn.Conv2d(features, 128, kernel_size=3, stride=1, padding=1),
            Interpolate(scale_factor=2, mode="bilinear"),
            nn.Conv2d(128, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 1, kernel_size=1, stride=1, padding=0),
            nn.ReLU(True) if non_negative else nn.Identity(),
        )

        if path:
            self.load(path)

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input data (image)

        Returns:
            tensor: depth
        """

        layer_1 = self.pretrained.layer1(x)
        layer_2 = self.pretrained.layer2(layer_1)
        layer_3 = self.pretrained.layer3(layer_2)
        layer_4 = self.pretrained.layer4(layer_3)

        layer_1_rn = self.scratch.layer1_rn(layer_1)
        layer_2_rn = self.scratch.layer2_rn(layer_2)
        layer_3_rn = self.scratch.layer3_rn(layer_3)
        layer_4_rn = self.scratch.layer4_rn(layer_4)

        path_4 = self.scratch.refinenet4(layer_4_rn)
        path_3 = self.scratch.refinenet3(path_4, layer_3_rn)
        path_2 = self.scratch.refinenet2(path_3, layer_2_rn)
        path_1 = self.scratch.refinenet1(path_2, layer_1_rn)

        out = self.scratch.output_conv(path_1)

        return torch.squeeze(out, dim=1)

ImportError: attempted relative import with no known parent package

In [ ]:
# Load the MiDaS model
def load_midas_model():
    model = DPTDepthModel(
        path="weights/dpt_large-midas-2f21e586.pt",
        backbone="vitl16_384",
        non_negative=True,
    )
    model.eval()  # Set the model to evaluation mode
    return model

# Preprocess the image
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform(image)

# Generate depth map using MiDaS
def generate_depth_map(model, image):
    with torch.no_grad():
        input_image = preprocess_image(image).unsqueeze(0)
        depth_map = model(input_image)
        depth_map = torch.nn.functional.interpolate(depth_map.unsqueeze(1), size=(384, 384), mode='bilinear', align_corners=False)
        depth_map = depth_map.squeeze().cpu().numpy()
    return depth_map


In [ ]:
# Capture frames and display depth map
def capture_and_display(midas_model):
    cap = cv2.VideoCapture(0)  # Start video capture from webcam

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Generate depth map
        depth_map = generate_depth_map(midas_model, frame)

        # Display depth map using Matplotlib
        plt.imshow(depth_map, cmap='plasma')
        plt.colorbar()
        plt.show(block=False)
        plt.pause(0.001)

        # Wait for 'q' key press to capture the next frame or exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            plt.close()  # Close the Matplotlib window
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    # Load MiDaS model
    midas_model = load_midas_model()

    # Capture and display depth map
    capture_and_display(midas_model)

if __name__ == "__main__":
    main()